In [1]:
from IPython import display
import pywikibot
from pywikibot import pagegenerators

enwp = pywikibot.Site('en', 'wikipedia')
commons = pywikibot.Site('commons', 'commons')
cat = pywikibot.Category(enwp, 'Category:Wikipedia files that shadow a file on Wikimedia Commons')


def commons_file_exists(cpage):
    """Check if the shadowed file still exists.
    
    The Commons file might have been deleted,
    or the enwiki file may have been moved without removing the template.
    Returns True if the file exists, False if it doesnt.
    """
    return cpage.exists()


def local_deletion_process(page):
    """Check if the enwiki file is awaiting deletion.
    
    Returns 'DR' if listed at FfD, 'DI' if a delayed speedy deletion,
    'SD' for speedy deletion, and '' for no deletion proceses.
    """
    # Enwiki adds "All ... " categories for deletable images, so we can just check for those
    SD_cats = (
        'Category:Candidates for speedy deletion',
        'Category:All Wikipedia files with the same name on Wikimedia Commons',
        'Category:All Wikipedia files with a different name on Wikimedia Commons')
        
    DI_cats = (
        'Category:All files proposed for deletion',
        'Category:All Wikipedia files with unknown source',
        'Category:All replaceable non-free use Wikipedia files',
        'Category:All orphaned non-free use Wikipedia files',
        'Category:All Wikipedia files with no non-free use rationale',
        'Category:All Wikipedia files with unknown copyright status',
        'Category:All Wikipedia files missing evidence of permission')
        
    DR_cats = ('Category:Wikipedia files for discussion')
    
    
    def check_cats(category):
        """Check if the given category is a known deletion category

        If it is, return a string.
        If it isn't, return an empty string.
        """
        title = category.title()
        if title in SD_cats:
            return 'SD'
        elif title in DR_cats:
            return 'DR'
        elif title in DI_cats:
            return 'DI'
        else:
            return ''
    
    
    for pagecat in page.categories():
        cc = check_cats(pagecat)
        if cc: return cc


def commons_deletion_process(cpage):
    """Check if the Commons file is awaiting deletion.
    
    Returns 'DR' if listed at DR, 'DI' if a delayed speedy deletion,
    'SD' for speedy deletion, and '' for no deletion proceses.
    """
    # Commons does not add "All..." categories, so we have to climb the cat tree recursively
    # Define deletion categories
    SD_cats = ('Category:Candidates for speedy deletion')
    DI_cats = (
        'Category:Media without a license',
        'Category:Media without a source',
        'Category:Media missing permission')
    DR_cats = ('Category:Deletion requests')
    
    def check_cats(category):
        """Check if the given category is a known deletion category
        
        If it is, return a string.
        If it isn't, return an empty string.
        """
        ctitle = category.title()
        if ctitle in SD_cats:
            return 'SD'
        elif ctitle in DR_cats:
            return 'DR'
        elif ctitle in DI_cats:
            return 'DI'
        else:
            return ''
        
        
    def recurse_check_cats(category, level):
        """Check if the category is a deletion category.
        
        First, check if this category is the CommonsRoot. If it is, we've reached the top
        of the category tree and this isn't a deletion category. Return an empty string
        Then, check if this category is a known deletion category.
        If it is, return a string.
        If it isn't, check the parent categories. 
        """
        ctitle = category.title()
        if (ctitle == 'Category:CommonsRoot') or level == 2:
            # Check if we're too high in the category tree
            # Deletion parents are only one or two categories above
            # the categories on the page
            return ''
        else:
            cc = check_cats(category)
            # If this is a deletion category, pass that up.
            if cc: return cc
            else:
                # If not, increment the level and check the parent categories
                for catcat in category.categories():
                    # If a parent is a deletion category, return such.
                    rcl = recurse_check_cats(catcat, level + 1)
                    if rcl: return rcl
          
    
    # Check just the page categories, as some deletions categorize in the base category
    for pagecat in cpage.categories():
        cc = check_cats(pagecat)
        if cc: return cc

    # If we haven't returned yet, we need to check the parent categories too.
    for pagecat in cpage.categories():
        rcc = recurse_check_cats(pagecat, 0)
        if rcc: return rcc
        

def gen_file_lists(ignore):
    """Run through the category and sort the files into lists. Return a tuple of lists"""
    no_commons_file = []
    enwp_deletion = []
    enwp_speedy = []
    commons_deletion = []
    commons_speedy = []
    shadowing = []
        
    for page in pagegenerators.CategorizedPageGenerator(cat):
        title = page.title()
        cpage = pywikibot.Page(commons, title)
        
        if title in ignore:
            continue
        elif commons_file_exists(cpage):
            cdp = commons_deletion_process(cpage)
            ldp = local_deletion_process(page)
                        
            if not (cdp or ldp):
                # If the file is not awaiting deletion anywhere, add to shadowing and end. (most common)
                # If cdp is empty and ldp is empty, file shadows.
                shadowing.append((page, cpage))
            else:
                if ldp == 'DR':
                    # If the file is awaiting deletion on enwp, add to list and end.
                    enwp_deletion.append((page, cpage))
                elif ldp == 'DI':
                    enwp_deletion.append((page, cpage))
                elif ldp == 'SD':
                    enwp_speedy.append((page, cpage))
                    
                if cdp == 'DR':
                    # If the file is awaiting deletion on Commons, add to list and continue
                    commons_deletion.append((page, cpage))
                elif cdp == 'DI':
                    commons_deletion.append((page, cpage))
                elif cdp == 'SD':
                    commons_speedy.append((page, cpage))
            
        else:
            # If the file is deleted on Commons, add to no_commons_file and end.
            no_commons_file.append((page, cpage))
            
    return (no_commons_file, enwp_deletion, enwp_speedy, commons_deletion, commons_speedy, shadowing)
                
                
def output_lists(lists):
    """Take the lists and render them into markdown"""
    
    output = '# {{Shadows Commons}}\n'
    
    def tomarkdown(list):
        md = ''
        for page, cpage in list:
            md += '* [{title}]({elink}) ([Commons]({clink}))\n'.format(
                title = page.title(), elink = page.full_url(),
                clink = cpage.full_url())
        return md
            
    
    no_commons_file, enwp_deletion, enwp_speedy, commons_deletion, commons_speedy, shadowing = lists
    
    output += '## Files with {{Shadows Commons}} but no Commons file\n'
    output += 'These files may have shadowed a Commons file at some point, but they don\'t  anymore.\n\n'
    output += tomarkdown(no_commons_file)
    
    output += '\n## Files that shadow Commons\n\n'
    output += tomarkdown(shadowing)
    
    output += '\n## Files awaiting deletion on the English Wikipedia\n'
    output += '### Speedy Deletion\n\n'
    output += tomarkdown(enwp_speedy)
    output += '### Delayed Deletion or FfD\n\n'
    output += tomarkdown(enwp_deletion)
    
    output += '\n## Files awaiting deletion on Commons\n'
    output += '### Speedy Deletion\n\n'
    output += tomarkdown(commons_speedy)
    output += '### Delayed Deletion or DR\n\n'
    output += tomarkdown(commons_deletion)
    return output
    
def __main__():
    ignore = []
    lists = gen_file_lists(ignore)
    md = output_lists(lists)

    display.display_markdown(md, raw=True)

In [2]:
__main__()

# {{Shadows Commons}}
## Files with {{Shadows Commons}} but no Commons file
These files may have shadowed a Commons file at some point, but they don't  anymore.

* [File:Pionono.jpg](https://en.wikipedia.org/wiki/File%3APionono.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3APionono.jpg))

## Files that shadow Commons

* [File:Allied Forces Northern Europe.jpg](https://en.wikipedia.org/wiki/File%3AAllied_Forces_Northern_Europe.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3AAllied_Forces_Northern_Europe.jpg))
* [File:Eén logo 2019.svg](https://en.wikipedia.org/wiki/File%3AE%C3%A9n_logo_2019.svg) ([Commons](https://commons.wikimedia.org/wiki/File%3AE%C3%A9n_logo_2019.svg))
* [File:Family Matters.jpg](https://en.wikipedia.org/wiki/File%3AFamily_Matters.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3AFamily_Matters.jpg))
* [File:Fashion District Philadelphia.jpg](https://en.wikipedia.org/wiki/File%3AFashion_District_Philadelphia.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3AFashion_District_Philadelphia.jpg))
* [File:Hikvision logo.svg](https://en.wikipedia.org/wiki/File%3AHikvision_logo.svg) ([Commons](https://commons.wikimedia.org/wiki/File%3AHikvision_logo.svg))
* [File:Japan pitch accent map.png](https://en.wikipedia.org/wiki/File%3AJapan_pitch_accent_map.png) ([Commons](https://commons.wikimedia.org/wiki/File%3AJapan_pitch_accent_map.png))
* [File:One Nashville Place.jpg](https://en.wikipedia.org/wiki/File%3AOne_Nashville_Place.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3AOne_Nashville_Place.jpg))
* [File:Porch.jpg](https://en.wikipedia.org/wiki/File%3APorch.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3APorch.jpg))
* [File:Queen's-Hall-1912.jpg](https://en.wikipedia.org/wiki/File%3AQueen%27s-Hall-1912.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3AQueen%27s-Hall-1912.jpg))
* [File:SMS Nassau NH 46827.jpg](https://en.wikipedia.org/wiki/File%3ASMS_Nassau_NH_46827.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3ASMS_Nassau_NH_46827.jpg))
* [File:SMS Nassau NH 46828.jpg](https://en.wikipedia.org/wiki/File%3ASMS_Nassau_NH_46828.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3ASMS_Nassau_NH_46828.jpg))
* [File:Southern Cassowary.jpg](https://en.wikipedia.org/wiki/File%3ASouthern_Cassowary.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3ASouthern_Cassowary.jpg))
* [File:SpongeBob SquarePants logo.svg](https://en.wikipedia.org/wiki/File%3ASpongeBob_SquarePants_logo.svg) ([Commons](https://commons.wikimedia.org/wiki/File%3ASpongeBob_SquarePants_logo.svg))
* [File:TG4 logo.svg](https://en.wikipedia.org/wiki/File%3ATG4_logo.svg) ([Commons](https://commons.wikimedia.org/wiki/File%3ATG4_logo.svg))

## Files awaiting deletion on the English Wikipedia
### Speedy Deletion

* [File:Air North logo.jpg](https://en.wikipedia.org/wiki/File%3AAir_North_logo.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3AAir_North_logo.jpg))
### Delayed Deletion or FfD

* [File:Arthur Chase.jpg](https://en.wikipedia.org/wiki/File%3AArthur_Chase.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3AArthur_Chase.jpg))
* [File:Changchun University of Science and Technology.jpg](https://en.wikipedia.org/wiki/File%3AChangchun_University_of_Science_and_Technology.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3AChangchun_University_of_Science_and_Technology.jpg))
* [File:Francisco Ascaso.jpg](https://en.wikipedia.org/wiki/File%3AFrancisco_Ascaso.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3AFrancisco_Ascaso.jpg))
* [File:Frank Guarrera.jpg](https://en.wikipedia.org/wiki/File%3AFrank_Guarrera.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3AFrank_Guarrera.jpg))
* [File:Kumbakonam.jpg](https://en.wikipedia.org/wiki/File%3AKumbakonam.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3AKumbakonam.jpg))
* [File:NovaM.png](https://en.wikipedia.org/wiki/File%3ANovaM.png) ([Commons](https://commons.wikimedia.org/wiki/File%3ANovaM.png))
* [File:Perfetti Van Melle logo.svg](https://en.wikipedia.org/wiki/File%3APerfetti_Van_Melle_logo.svg) ([Commons](https://commons.wikimedia.org/wiki/File%3APerfetti_Van_Melle_logo.svg))
* [File:Qishlah.jpg](https://en.wikipedia.org/wiki/File%3AQishlah.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3AQishlah.jpg))
* [File:Rheobatrachus silus.jpg](https://en.wikipedia.org/wiki/File%3ARheobatrachus_silus.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3ARheobatrachus_silus.jpg))
* [File:San Nicolo.jpg](https://en.wikipedia.org/wiki/File%3ASan_Nicolo.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3ASan_Nicolo.jpg))
* [File:Sarto.jpg](https://en.wikipedia.org/wiki/File%3ASarto.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3ASarto.jpg))
* [File:Troodon.jpg](https://en.wikipedia.org/wiki/File%3ATroodon.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3ATroodon.jpg))
* [File:Tsinghua University Logo.svg](https://en.wikipedia.org/wiki/File%3ATsinghua_University_Logo.svg) ([Commons](https://commons.wikimedia.org/wiki/File%3ATsinghua_University_Logo.svg))
* [File:Well of Souls.jpg](https://en.wikipedia.org/wiki/File%3AWell_of_Souls.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3AWell_of_Souls.jpg))

## Files awaiting deletion on Commons
### Speedy Deletion

### Delayed Deletion or DR

* [File:Alexander Dennis logo.svg](https://en.wikipedia.org/wiki/File%3AAlexander_Dennis_logo.svg) ([Commons](https://commons.wikimedia.org/wiki/File%3AAlexander_Dennis_logo.svg))
* [File:Comic.jpg](https://en.wikipedia.org/wiki/File%3AComic.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3AComic.jpg))
* [File:Deliveroo logo.svg](https://en.wikipedia.org/wiki/File%3ADeliveroo_logo.svg) ([Commons](https://commons.wikimedia.org/wiki/File%3ADeliveroo_logo.svg))
* [File:Groupe SEB logo.svg](https://en.wikipedia.org/wiki/File%3AGroupe_SEB_logo.svg) ([Commons](https://commons.wikimedia.org/wiki/File%3AGroupe_SEB_logo.svg))
* [File:Kizuna AI.png](https://en.wikipedia.org/wiki/File%3AKizuna_AI.png) ([Commons](https://commons.wikimedia.org/wiki/File%3AKizuna_AI.png))
* [File:Maldivian logo.png](https://en.wikipedia.org/wiki/File%3AMaldivian_logo.png) ([Commons](https://commons.wikimedia.org/wiki/File%3AMaldivian_logo.png))
* [File:Najibullah.jpg](https://en.wikipedia.org/wiki/File%3ANajibullah.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3ANajibullah.jpg))
* [File:Ramón Mercader.jpg](https://en.wikipedia.org/wiki/File%3ARam%C3%B3n_Mercader.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3ARam%C3%B3n_Mercader.jpg))
* [File:Terrys.jpg](https://en.wikipedia.org/wiki/File%3ATerrys.jpg) ([Commons](https://commons.wikimedia.org/wiki/File%3ATerrys.jpg))
* [File:Windows 2000 logo.svg](https://en.wikipedia.org/wiki/File%3AWindows_2000_logo.svg) ([Commons](https://commons.wikimedia.org/wiki/File%3AWindows_2000_logo.svg))


### Licensing
Copyright 2019 AntiCompositeNumber

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

  http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

### To fork:
1. Open the [raw page](?format=raw) and save it to your computer
2. Go to your [PAWS control panel](https://paws.wmflabs.org/paws/hub) and sign in using OAUTH
3. Click Upload and upload the file from step 1
4. To run, open the notebook and click Cell > Run All